In [1]:
import os
import re
from datetime import datetime
from pathlib import Path
import datetime
import folium
import shapely
import json
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd

In [2]:
DATA_PATH = Path('/home/io/ASTROSAT/code/data-team/notebooks/rice_paddy')

### add username and password

In [3]:
user = os.environ['user']
password = os.environ['password']
api_url = "https://scihub.copernicus.eu/dhus/"

In [4]:
# Sentinel-1 attributes
s1_attr = {'product':{'S':'SLC', 'G':'GRD','O':'OCN'},
           'orbit':{'A':'ascending', 'D':'descending'}
          }

### search for scenes

In [5]:
# search by polygon in Geojson format
footprint = geojson_to_wkt(read_geojson(DATA_PATH.joinpath('aoi','MADA.geojson')))
# api query
api = SentinelAPI(
    user=user,
    password=password,
    api_url= api_url 
)
# perform query based on date
products = api.query(footprint,
                     date=('20190201', '20190816'), 
                     platformname='Sentinel-1',
                     producttype=s1_attr['product']['G'],
                     orbitdirection=s1_attr['orbit']['A'],
                     #relativeOrbitNumber=164
                    )
print (f'{len(products)} scenes have been found')
    

32 scenes have been found


In [6]:
# convert to Pandas DataFrame
products_df = api.to_geodataframe(products)
products_df[:3]

/home/io/.local/share/virtualenvs/rice_paddy-iFSoDga7/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,title,link,link_alternative,link_icon,summary,beginposition,endposition,ingestiondate,missiondatatakeid,slicenumber,orbitnumber,lastorbitnumber,relativeorbitnumber,lastrelativeorbitnumber,sensoroperationalmode,swathidentifier,orbitdirection,producttype,timeliness,platformname,platformidentifier,instrumentname,instrumentshortname,filename,format,productclass,polarisationmode,acquisitiontype,status,size,identifier,uuid,geometry
e3752352-0dcd-48b1-83cf-1e5af1bdc6d3,S1A_IW_GRDH_1SDV_20190809T113445_20190809T1135...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-08-09T11:34:45.912Z, Instrument: SA...",2019-08-09 11:34:45.912,2019-08-09 11:35:10.909,2019-08-09 16:29:27.507,211068,8,28492,28492,70,70,IW,IW,ASCENDING,GRD,Fast-24h,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S1A_IW_GRDH_1SDV_20190809T113445_20190809T1135...,SAFE,S,VV VH,NOMINAL,ARCHIVED,1.61 GB,S1A_IW_GRDH_1SDV_20190809T113445_20190809T1135...,e3752352-0dcd-48b1-83cf-1e5af1bdc6d3,"MULTIPOLYGON (((99.28992 4.40835, 101.56293 4...."
03386767-c73d-44a7-bf88-bd38491fde15,S1A_IW_GRDH_1SDV_20190809T113510_20190809T1135...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-08-09T11:35:10.911Z, Instrument: SA...",2019-08-09 11:35:10.911,2019-08-09 11:35:35.910,2019-08-09 16:29:23.947,211068,9,28492,28492,70,70,IW,IW,ASCENDING,GRD,Fast-24h,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S1A_IW_GRDH_1SDV_20190809T113510_20190809T1135...,SAFE,S,VV VH,NOMINAL,ARCHIVED,1.61 GB,S1A_IW_GRDH_1SDV_20190809T113510_20190809T1135...,03386767-c73d-44a7-bf88-bd38491fde15,"MULTIPOLYGON (((98.97773 5.92069, 101.25681 6...."
4f87519e-9e8d-4210-a7d1-e21699505552,S1A_IW_GRDH_1SDV_20190728T113444_20190728T1135...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-07-28T11:34:44.949Z, Instrument: SA...",2019-07-28 11:34:44.949,2019-07-28 11:35:09.948,2019-07-28 15:08:35.812,209681,8,28317,28317,70,70,IW,IW,ASCENDING,GRD,Fast-24h,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S1A_IW_GRDH_1SDV_20190728T113444_20190728T1135...,SAFE,S,VV VH,NOMINAL,ARCHIVED,1.61 GB,S1A_IW_GRDH_1SDV_20190728T113444_20190728T1135...,4f87519e-9e8d-4210-a7d1-e21699505552,"MULTIPOLYGON (((99.29038 4.40817, 101.56340 4...."


### Product metadata

In [7]:
uuid = products_df['uuid'][0]
product_meta = api.get_product_odata(uuid, full=True)

### Folium - add data on the map

In [13]:
aoi = DATA_PATH.joinpath('aoi','MADA.geojson')

In [14]:
gdf = gpd.read_file(aoi)

#### Convert shapefile to Geojson

In [15]:
# FIX the error: datetime.datetime(...) is not JSON serializable
# https://code-maven.com/serialize-datetime-object-as-json-in-python
def myconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
    
scenes = json.dumps(shapely.geometry.mapping(products_df), default = myconverter)

In [16]:
# Locate AOI on the map
location = [6.10, 100.3]
m = folium.Map(location=location,
               zoom_start=8,
               name='Basemap'
              )

# overlay Sentinel-1 scenes
folium.GeoJson(
    scenes,
    name='Sentinel-1 scenes',
    # polygon styling: https://gist.github.com/wrobstory/5609786
    style_function= lambda x :{'fillColor':'grey', 
                               'fillOpacity': 0.3, 
                               'color':'black',
                               'weight': 1
                              }
).add_to(m)


# overlay our AOI
folium.GeoJson(
    gdf,
    name='MADA Region',
).add_to(m)


folium.map.LayerControl('topright', 
                        collapsed=False
).add_to(m)
m

### Check if the prodcut is online or a [Long Term Archive (LTA)](https://scihub.copernicus.eu/userguide/LongTermArchive)
- Download products acquired in specific days

In [7]:
for i, row in products_df.iterrows():
    product = row[['title','uuid','beginposition']]
    # convert datatime to string
    date = product['beginposition']
    date_str = date.strftime('%Y-%m-%d')
    uuid = product['uuid']
    product_meta = api.get_product_odata(uuid)
    if product_meta['Online']:
        print('Product {} is online.'.format(product_meta['title']))
        # if online, download it
        #api.download(product['uuid'], DATA_PATH.joinpath('output'))
    else:
        print('Product {} is not online.'.format(product_meta['title']))
        
#     # specify speciftc date to download
#     dates = ['2019-09-20','2019-09-08']
#     for d in dates:
#         if d in date_str:
#             # get product metadata
#             meta = api.get_product_odata(uuid)
#             if meta['Online']:
#                 print('Product {} is online.'.format(meta['title']))
#                 # if online, download it
#                 #api.download(product['uuid'], DATA_PATH.joinpath('output'))
#             else:
#                 print('Product {} is not online.'.format(meta['title']))

Product S1A_IW_GRDH_1SDV_20190216T230258_20190216T230323_025961_02E462_02EF is not online.
Product S1A_IW_GRDH_1SDV_20190216T230323_20190216T230348_025961_02E462_9A02 is not online.
Product S1A_IW_GRDH_1SDV_20190204T230323_20190204T230348_025786_02DE30_9862 is not online.
Product S1A_IW_GRDH_1SDV_20190204T230258_20190204T230323_025786_02DE30_244E is not online.
Product S1A_IW_GRDH_1SDV_20190123T230324_20190123T230349_025611_02D7D5_2EB3 is not online.
Product S1A_IW_GRDH_1SDV_20190123T230259_20190123T230324_025611_02D7D5_8FAD is not online.
Product S1A_IW_GRDH_1SDV_20190111T230324_20190111T230349_025436_02D172_28AB is not online.
Product S1A_IW_GRDH_1SDV_20190111T230259_20190111T230324_025436_02D172_48DB is not online.
